In [43]:
import torch
import re
import os
import json
import pickle
import openai
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm
import ast
from dotenv import load_dotenv

In [2]:
load_dotenv()
openai.api_key = os.environ['OPENAI_API']

In [27]:
def extract_present_traits(species, df):

    s = df.loc[species]
    GT_traits = list(s.where(s == 1).dropna().index)
    empty_traits = [F"{Entity}:" for (Entity, Value) in GT_traits]

    return GT_traits, empty_traits

### Trait Dicts

In [3]:
folder_traits = "../../../data/OpenAI/Traits/"

with open(F"{folder_traits}Andrei.json", 'r') as f:
  caribbean_traits_dict = json.load(f)

with open(F"{folder_traits}Andrei.json", 'r') as f:
  caribbean_traits_dict = json.load(f)

with open(F"{folder_traits}Andrei.json", 'r') as f:
  caribbean_traits_dict = json.load(f)

### DataFrames

In [4]:
root = "../../../data/OpenAI/DataFrames/"

file = "DF_Andrei.csv"
df_Andrei = pd.read_csv(F"{root}{file}", header=[0, 1], index_col=0)\
    .rename_axis('Species', axis='index')
df_Andrei_species = list(df_Andrei.index)


file = "DF_Daniel.csv"
df_Daniel = pd.read_csv(F"{root}{file}", header=[0, 1], index_col=0)\
    .rename_axis('Species', axis='index')\
    .dropna()
df_Daniel_species = list(df_Daniel.index)

file = "DF_Pierre.csv"
df_Pierre = pd.read_csv(F"{root}{file}", header=[0, 1], index_col=0)\
    .rename_axis('Species', axis='index')
df_Pierre_species = list(df_Pierre.index)

In [5]:
species = df_Andrei.index[0]
species

'Avicennia germinans'

In [6]:
for trait in caribbean_traits_dict.keys():
    trait
    # print(trait)

traits = list(caribbean_traits_dict.keys())
traits

['Life form',
 'Leaf position',
 'Leaf composition',
 'Leaf shape',
 'Leaf margin',
 'Leaf upper side',
 'Leaf lower side',
 'Leaf glands',
 'Leaf rachis',
 'Thorns/spines',
 'Stipules',
 'Inflorescence type',
 'Sepals / calyx shape',
 'Sepals / calyx numer',
 'Petals / corolla shape',
 'Petals / corolla number',
 'Petals / corolla colour',
 'Stamen shape',
 'Stamen number',
 'Fruit type',
 'Fruit shape',
 'Fruit colour',
 'Aril colour',
 'Seed colour']

In [21]:
question = F"Can you create semantic triples for the following traits {traits}, for the species '{species}'? \nCan you use the example format I provided:\n"
# print(question)

example = (F"http://dbpedia.org/resource/{species.replace(' ', '_')}", "http://dbpedia.org/ontology/leafPosition", "alternate")
# print(example)

content = F"{question} {example}"
print(content)

Can you create semantic triples for the following traits ['Life form', 'Leaf position', 'Leaf composition', 'Leaf shape', 'Leaf margin', 'Leaf upper side', 'Leaf lower side', 'Leaf glands', 'Leaf rachis', 'Thorns/spines', 'Stipules', 'Inflorescence type', 'Sepals / calyx shape', 'Sepals / calyx numer', 'Petals / corolla shape', 'Petals / corolla number', 'Petals / corolla colour', 'Stamen shape', 'Stamen number', 'Fruit type', 'Fruit shape', 'Fruit colour', 'Aril colour', 'Seed colour'], for the species 'Avicennia germinans'? 
Can you use the example format I provided:
 ('http://dbpedia.org/resource/Avicennia_germinans', 'http://dbpedia.org/ontology/leafPosition', 'alternate')


In [22]:
messages = [
    # {"role": "assistant", "content": text},
    {"role": "user", "content": content}
    ]
# Call the ChatGPT API to generate a completion for the prompt
completion = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = messages,
)

In [23]:
completion

<OpenAIObject chat.completion id=chatcmpl-6uzSI32T8VCvQd2Byoh0gQ6hYCeCI at 0x7fd988727bf0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "\n\n('http://dbpedia.org/resource/Avicennia_germinans', 'http://dbpedia.org/ontology/lifeForm', 'tree'),\n('http://dbpedia.org/resource/Avicennia_germinans', 'http://dbpedia.org/ontology/leafPosition', 'alternate'),\n('http://dbpedia.org/resource/Avicennia_germinans', 'http://dbpedia.org/ontology/leafComposition', 'simple'),\n('http://dbpedia.org/resource/Avicennia_germinans', 'http://dbpedia.org/ontology/leafShape', 'lanceolate'),\n('http://dbpedia.org/resource/Avicennia_germinans', 'http://dbpedia.org/ontology/leafMargin', 'entire'),\n('http://dbpedia.org/resource/Avicennia_germinans', 'http://dbpedia.org/ontology/leafUpperSide', 'green'),\n('http://dbpedia.org/resource/Avicennia_germinans', 'http://dbpedia.org/ontology/leafLowerSide', 'gray'),\n('http://dbpedia.org/resource/Avic

In [41]:
GT = extract_present_traits(species, df_Andrei)[0]
GT

[('Life form', 'tree'),
 ('Leaf position', 'opposite'),
 ('Leaf composition', 'entire'),
 ('Leaf shape', 'lanceolate, elliptic'),
 ('Leaf margin', 'entire'),
 ('Leaf upper side', 'salt crystals'),
 ('Leaf lower side', 'powdery'),
 ('Leaf glands', 'absent'),
 ('Thorns/spines', 'absent'),
 ('Stipules', 'absent'),
 ('Inflorescence type', 'panicle'),
 ('Sepals / calyx shape', 'free'),
 ('Sepals / calyx numer', '5'),
 ('Petals / corolla shape', 'tubular'),
 ('Petals / corolla number', '4'),
 ('Petals / corolla colour', 'white'),
 ('Stamen shape', 'longer than corolla'),
 ('Stamen number', '4'),
 ('Fruit type', 'capsule'),
 ('Fruit shape', 'flattened ovate'),
 ('Fruit colour', 'green'),
 ('Seed colour', 'green')]

In [45]:
res = completion['choices'][0]['message']['content']
res

list_of_tuples = ast.literal_eval("[" + res.replace("\n", "") + "]")
# list_of_tuples

In [49]:
for r, g in zip(list_of_tuples, GT):
    print(F"GPT output: {r[1:]}")
    print(F"GT Data:    {g}")

GPT output: ('http://dbpedia.org/ontology/lifeForm', 'tree')
GT Data:    ('Life form', 'tree')
GPT output: ('http://dbpedia.org/ontology/leafPosition', 'alternate')
GT Data:    ('Leaf position', 'opposite')
GPT output: ('http://dbpedia.org/ontology/leafComposition', 'simple')
GT Data:    ('Leaf composition', 'entire')
GPT output: ('http://dbpedia.org/ontology/leafShape', 'lanceolate')
GT Data:    ('Leaf shape', 'lanceolate, elliptic')
GPT output: ('http://dbpedia.org/ontology/leafMargin', 'entire')
GT Data:    ('Leaf margin', 'entire')
GPT output: ('http://dbpedia.org/ontology/leafUpperSide', 'green')
GT Data:    ('Leaf upper side', 'salt crystals')
GPT output: ('http://dbpedia.org/ontology/leafLowerSide', 'gray')
GT Data:    ('Leaf lower side', 'powdery')
GPT output: ('http://dbpedia.org/ontology/leafGlands', 'absent')
GT Data:    ('Leaf glands', 'absent')
GPT output: ('http://dbpedia.org/ontology/leafRachis', 'winged')
GT Data:    ('Thorns/spines', 'absent')
GPT output: ('http://dbpe

In [ ]:
# # Define the path to the directory where the prompts and results will be saved
# folder_prompts = "../../../data/OpenAI/PromptsResults/"

# # Loop over each JSON file in the list caribbean_jsons_paras
# for json_file in caribbean_jsons_paras[1:2]:

#     try:
#         # Try to read in the JSON file
#         with open(json_file, 'r') as f:
#             caribbean_species_paragraph = json.load(f)
        
#         # Get the name of the species from the keys of the dictionary in the JSON file
#         species = list(caribbean_species_paragraph.keys())[0]
#     except:
#         # If the JSON file cannot be read in, get the name of the species from the file name
#         species = json_file[62:-29].replace('_', ' ')
#         continue
    
#     # Join the paragraphs for the species into a single text string
#     text = " ".join(caribbean_species_paragraph[species])

#     # Replace spaces in the species name with underscores
#     folder_species = species.replace(' ', '_')

#     # Try to create a directory for the prompts for the species
#     try:
#         os.makedirs(F"{folder_prompts}{folder_species}")
#     except FileExistsError:
#         pass

#     # Loop over each trait and trait options in the caribbean_traits_dict dictionary
#     for trait, trait_options in (pbar := tqdm(caribbean_traits_dict.items(), leave=False, position=0)):
#         pbar.set_description(f"{species}")

#         # Create the question and options for the ChatGPT prompt
#         # question = F"Which of the following values correctly describe(s) the '{trait}' trait mentioned in the text? Please select all that apply."
#         # options = trait_options + ["None of the above", "Someting else (please specify)"]
#         # user_content = F"Question: {question}  \n\nValues: {options}"

#         question = F"Which of the following values correctly describe(s) the '{trait}' trait mentioned in the text? If none of the following values apply, please select 'None of the above'. Please select all that apply:" 
#         options = trait_options + ["Other: description of trait not mentioned in the list."]
#         options = F"{options}."
#         user_content = F"{question} {options}"

#         # print(user_content)

#         # Create the messages to send to the ChatGPT API
#         messages = [
#             {"role": "assistant", "content": text},
#             {"role": "user", "content": user_content}
#             ]
#         # Call the ChatGPT API to generate a completion for the prompt
#         completion = openai.ChatCompletion.create(
#             model = "gpt-3.5-turbo",
#             messages = messages,
#         )

#         # Combine the words in the trait name with capital letters and use this as the file name
#         file_name = combine_words_with_capital(trait)
#         # Save the completion to a JSON file with the file name in the species directory
#         with open(F"{folder_prompts}{folder_species}/{file_name}.json", 'w') as fp:
#             json.dump(completion, fp)
            
